In [129]:
# Install necessary packages
!pip install -q amplpy ampltools

# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# Instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL()  # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [130]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
# Load data from the updated sheets
annual_hours = pd.read_excel("/content/drive/MyDrive/645/realignment_data.xlsx", sheet_name='StoreData1')

annual_trips = pd.read_excel("/content/drive/MyDrive/645/realignment_data.xlsx", sheet_name='StoreData2')

regional_office_data = pd.read_excel("/content/drive/MyDrive/645/realignment_data.xlsx", sheet_name='Regional Office Data')

mileage = pd.read_excel("/content/drive/MyDrive/645/realignment_data.xlsx", sheet_name='TravelData1')

travel_time = pd.read_excel("/content/drive/MyDrive/645/realignment_data.xlsx", sheet_name='TravelData2')

# Replace '--' in travel data with a high value (e.g., 9999)
mileage.replace('--', 9999, inplace=True)
mileage.iloc[:, 1:] = mileage.iloc[:, 1:].apply(pd.to_numeric)

# Replace '--' in travel data with a high value (e.g., 9999)
travel_time.replace('--', 9999, inplace=True)
travel_time.iloc[:, 1:] = travel_time.iloc[:, 1:].apply(pd.to_numeric)

# Process "Annual Hours" sheet (StoreData1)
hours = annual_hours.set_index('Store Name').stack().to_dict()

# Process "Annual Trips" sheet (StoreData2)
trips = annual_trips.set_index('Store Name').stack().to_dict()

# Process "Regional Office Data" sheet
availability = regional_office_data.set_index('Office Name').stack().to_dict()

# Process "Mileage" sheet (TravelData1)
mileage = mileage.set_index('Store Name').stack().to_dict()

# Process "Travel Time" sheet (TravelData2)
traveltime = travel_time.set_index('Store Name').stack().to_dict()

StateMileageRate=0.585
HourlyWage=26

<ipython-input-131-3deddc5566bf>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mileage.replace('--', 9999, inplace=True)
<ipython-input-131-3deddc5566bf>:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  travel_time.replace('--', 9999, inplace=True)


In [132]:
#Print the dictionaries to verify
print("Annual Trips Dictionary:", trips)
print("Hours Required Dictionary:", hours)
print("Regional Office Data Dictionary:", availability)
print("Mileage Dictionary:", mileage)
print("Travel Time Dictionary:", traveltime)

Annual Trips Dictionary: {('Albemarle_County', 'Inventory'): 39, ('Albemarle_County', 'Payroll'): 133, ('Albemarle_County', 'Hiring'): 0, ('Albemarle_County', 'Marketing'): 19, ('Albemarle_County', 'Merchandising'): 5, ('Amherst_County', 'Inventory'): 0, ('Amherst_County', 'Payroll'): 11, ('Amherst_County', 'Hiring'): 0, ('Amherst_County', 'Marketing'): 1, ('Amherst_County', 'Merchandising'): 0, ('Augusta_County', 'Inventory'): 92, ('Augusta_County', 'Payroll'): 91, ('Augusta_County', 'Hiring'): 186, ('Augusta_County', 'Marketing'): 30, ('Augusta_County', 'Merchandising'): 30, ('Buckingham_County', 'Inventory'): 2, ('Buckingham_County', 'Payroll'): 2, ('Buckingham_County', 'Hiring'): 23, ('Buckingham_County', 'Marketing'): 15, ('Buckingham_County', 'Merchandising'): 4, ('Caroline_County', 'Inventory'): 4, ('Caroline_County', 'Payroll'): 17, ('Caroline_County', 'Hiring'): 6, ('Caroline_County', 'Marketing'): 5, ('Caroline_County', 'Merchandising'): 4, ('Charles_City_County', 'Inventory'

In [133]:
Speed=50
# Store names and office names
stores = ['Albemarle_County', 'Amherst_County', 'Augusta_County', 'Buckingham_County',
          'Caroline_County', 'Charles_City_County', 'Chesterfield_County',
          'City_of_Fredericksburg', 'City_of_Richmond', 'Culpeper_County',
          'Cumberland_County', 'Dinwiddie_County', 'Essex_County', 'Fauquier_County',
          'Fluvanna_County', 'Goochland_County', 'Greene_County', 'Hanover_County',
          'Henrico_County', 'Hopewell_County', 'James_City_County', 'King_and_Queen_County',
          'King_George_County', 'King_William_County', 'Louisa_County', 'Madison_County',
          'Mathews_County', 'Nelson_County', 'New_Kent_County', 'Orange_County', 'Page_County',
          'Powhatan_County', 'Prince_George_County', 'Prince_William_County', 'Rappahannock_County',
          'Rockbridge_County', 'Rockingham_County', 'Shenandoah_County', 'Spotsylvania_County',
          'Stafford_County', 'Warren_County', 'Westmoreland_County', 'York_County']

offices = ['Richmond', 'Staunton', 'Warrenton', 'Tappahannock']
tasks = ['Inventory','Payroll','Hiring','Marketing','Merchandising']

# Create indices for stores and offices
store_indices = {store: i for i, store in enumerate(stores)}
office_indices = {office: j for j, office in enumerate(offices)}

travelcost = {
    (store_indices[store], office_indices[office]): mileage.get((store, office), 0) * StateMileageRate
    for store in stores
    for office in offices
}

salarycost = {
    (store_indices[store], office_indices[office]): (traveltime.get((store, office), 0) / Speed) * HourlyWage
    for store in stores
    for office in offices
}

In [134]:
ampl.eval('''
reset;

# Define Sets
set STORES;
set OFFICES;
set TASKS;

# Parameters
param HoursRequired {STORES, TASKS};
param Trips {STORES, TASKS};
param HoursAvailable {OFFICES, TASKS};
param Mileage {STORES, OFFICES};
param TravelTime {STORES, OFFICES};
param StateMileageRate;
param HourlyWage;

# Cost Parameters
param TravelCost {i in STORES, j in OFFICES};
param SalaryCost {i in STORES, j in OFFICES};

# Decision Variables
var x {i in STORES, j in OFFICES} binary;

# Objective: Minimize the total cost (travel + salary cost)
minimize total_cost:
    sum {i in STORES, j in OFFICES} (TravelCost[i,j] + SalaryCost[i,j]) * x[i,j];

# Constraints:
# 1. Each store must be assigned to exactly one office
s.t. store_assignment {i in STORES}:
    sum {j in OFFICES} x[i,j] = 1;

# 2. The office must have enough available hours for each task
s.t. available_hours {j in OFFICES, k in TASKS}:
    sum {i in STORES} HoursRequired[i,k] * x[i,j] <= HoursAvailable[j,k];
''')

In [135]:
ampl.set['STORES'] = stores;
ampl.set['OFFICES'] = offices;
ampl.set['TASKS'] = tasks

# Set the parameters for the AMPL model
# 1. HoursRequired (stores x tasks)
ampl.param['HoursRequired'] = hours

# 2. Trips (stores x offices)
ampl.param['Trips'] = trips

# 3. HoursAvailable (offices x tasks)
ampl.param['HoursAvailable'] = availability

# 4. Mileage (stores x offices)
ampl.param['Mileage'] = mileage

# 5. TravelTime (stores x offices)
ampl.param['TravelTime'] = traveltime

# 6. StateMileageRate (fixed parameter)
ampl.param['StateMileageRate'] = StateMileageRate

# 7. HourlyWage (fixed parameter)
ampl.param['HourlyWage'] = HourlyWage
# Correct TravelCost dictionary with store and office names as keys
travelcost = {
    (store, office): mileage.get((store, office), 0) * StateMileageRate
    for store in stores
    for office in offices
}

# Now pass this directly to AMPL
ampl.param['TravelCost'] = travelcost

# Salary Cost Calculation with store names and office names as keys
salarycost = {
    (store, office): (traveltime.get((store, office), 0) / Speed) * HourlyWage
    for store in stores
    for office in offices
}

# Pass SalaryCost parameter to AMPL
ampl.param['SalaryCost'] = salarycost

In [136]:
ampl.eval('''expand;''')

minimize total_cost:
	41.5844*x['Albemarle_County','Richmond'] + 
	21.9882*x['Albemarle_County','Staunton'] + 
	42.3462*x['Albemarle_County','Warrenton'] + 
	65.39*x['Albemarle_County','Tappahannock'] + 
	71.292*x['Amherst_County','Richmond'] + 
	33.3216*x['Amherst_County','Staunton'] + 
	11048.9*x['Amherst_County','Warrenton'] + 
	11048.9*x['Amherst_County','Tappahannock'] + 
	11048.9*x['Augusta_County','Richmond'] + 
	71.8562*x['Augusta_County','Warrenton'] + 
	11048.9*x['Augusta_County','Tappahannock'] + 
	42.926*x['Buckingham_County','Richmond'] + 
	39.286*x['Buckingham_County','Staunton'] + 
	64.4436*x['Buckingham_County','Warrenton'] + 
	11048.9*x['Buckingham_County','Tappahannock'] + 
	24.9756*x['Caroline_County','Richmond'] + 
	11048.9*x['Caroline_County','Staunton'] + 
	37.024*x['Caroline_County','Warrenton'] + 
	22.6564*x['Caroline_County','Tappahannock'] + 
	22.0506*x['Charles_City_County','Richmond'] + 
	11048.9*x['Charles_City_County','Staunton'] + 
	11048.9*x['Charles_Cit

In [137]:
ampl.setOption('solver', 'cplex')

In [138]:
# Solve the problem
ampl.solve()

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 879.749
5 simplex iterations


In [139]:
# Display results
print("Optimal Total Cost:", ampl.getObjective("total_cost").value())
x = ampl.getVariable("x")
print("\nOptimal Assignments:")

# Use .toDict() to convert results to a dictionary
results = x.getValues().toDict()

# Iterate through the results
for (store, office), value in results.items():
    if value > 0.5:  # Check if binary decision variable is "selected"
        print(f"{store} assigned to {office}")

Optimal Total Cost: 879.7489999999999

Optimal Assignments:
Albemarle_County assigned to Staunton
Amherst_County assigned to Staunton
Augusta_County assigned to Staunton
Buckingham_County assigned to Staunton
Caroline_County assigned to Tappahannock
Charles_City_County assigned to Richmond
Chesterfield_County assigned to Richmond
City_of_Fredericksburg assigned to Tappahannock
City_of_Richmond assigned to Richmond
Culpeper_County assigned to Warrenton
Cumberland_County assigned to Richmond
Dinwiddie_County assigned to Richmond
Essex_County assigned to Tappahannock
Fauquier_County assigned to Warrenton
Fluvanna_County assigned to Staunton
Goochland_County assigned to Richmond
Greene_County assigned to Staunton
Hanover_County assigned to Richmond
Henrico_County assigned to Richmond
Hopewell_County assigned to Richmond
James_City_County assigned to Richmond
King_George_County assigned to Tappahannock
King_William_County assigned to Tappahannock
King_and_Queen_County assigned to Tappahanno